# Dynamic mode decomposition

Here we present the basic ideas behind a family of dynamic mode decomposition (DMD) methods. In particular the original "standard DMD"<cite data-cite="nbdmd-schmid2010dynamic">(Schmid, 2010)</cite>, its variant "exact DMD"<cite data-cite="nbdmd-tu2013dynamic">(Tu, 2013)</cite>, as well as "extended DMD" <cite data-cite="nbdmd-williams2015data">(Williams, 2015)</cite>. The principle idea is that DMD should be able to characterize / decompose potentially highly nonlinear dynamics by solving linear systems of equations and studying their spectral properties.

## Standard and exact DMD

For people already familiar with the method and interface, the [API docs](../api/generated/deeptime.decomposition.DMD.rst).

Considering a sequential set of data vectors $\{z_1,\ldots,z_m\}$ with $z_i\in\mathbb{R}^n$ and assuming that 

$$ z_{k+1} = Az_k $$

for some usually unknown matrix $A$, one can define standard DMD as follows (following the presentation of <cite data-cite="nbdmd-tu2013dynamic">(Tu, 2013)</cite>, originally introduced as DMD in <cite data-cite="nbdmd-schmid2010dynamic">(Schmid, 2010)</cite>).

1. The data is arranged into matrices $X, Y$ of matching pairs of time-shifted datapoints, i.e., $Y_{i+1} = X_{i}$ for $i \geq 1$.
2. Compute the (optionally truncated) SVD of $X$ to obtain $X = U\Sigma V^*$.
3. Define $\tilde{A} = U^*YV\Sigma^{-1}$.
4. Compute eigenvalues and eigenvectors $\tilde A w = \lambda w$.
5. The DMD mode corresponding to the DMD eigenvalue $\lambda$ is given by $\hat\varphi = Uw$.

It should be remarked that even if the data-generating dynamics are nonlinear, it is assumed that there exists some linear operator $A$ which approximates these.

The notion of "exact" DMD <cite data-cite="nbdmd-tu2013dynamic">(Tu, 2013)</cite> weakens the assumptions and only requires that the data vectors are corresponding pairs of data. The operator $A$ is defined as the solution to the best-approximation of $\| AX - Y\|_F$.
This leads to exact DMD modes corresponding to eigenvalue $\lambda$ of the form

$$ \varphi = \frac{1}{\lambda}YV\Sigma^{-1}w. $$

Both estimation modes are united within the same estimator in Deeptime:

In [ ]:
import deeptime as dt

standard_dmd = dt.decomposition.DMD(mode='standard')
exact_dmd = dt.decomposition.DMD(mode='exact')

The standard DMD is sometimes also referred to as projected DMD <cite data-cite="nbdmd-tu2013dynamic">(Tu, 2013)</cite> since the modes $\hat\varphi$ are eigenvectors of $\mathbb{P}_XA$, where $\mathbb{P}_X = UU^*$ is the orthogonal projection onto the image of $X$. If data vectors $y_i$ are in the $\mathrm{span}\{x_i\}$, then $\mathbb{P}_X = \mathrm{Id}$; projected/standard and exact DMD modes coincide.

This can be especially relevant if there are more dimensions than datapoints.

We demonstrate the methods using a random orthogonal matrix $A$ which serves as out propagator plus some normal distributed noise, i.e.,

$$ x_{t+1} = Ax_t + \mathcal{N}(0, 10^{-4}). $$

In [ ]:
import numpy as np

state = np.random.RandomState(45)

A = np.linalg.qr(state.normal(size=(2, 2)))[0]
x0 = state.uniform(-1, 1, size=(A.shape[0],))

data = np.empty((500, A.shape[0]))
data[0] = x0
for t in range(1, len(data)):
    data[t] = A @ data[t - 1] + state.normal(scale=1e-4, size=(2,))

Visualizing the generated data we can see that it is a jumping process between two wells.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(*data.T);

Fitting the models to this dataset reveals that the eigenvalues are identical:

In [ ]:
standard_model = standard_dmd.fit((data[:-1], data[1:])).fetch_model()
exact_model = exact_dmd.fit((data[:-1], data[1:])).fetch_model()

In [ ]:
print(f"Eigenvalues standard {standard_model.eigenvalues}, exact {exact_model.eigenvalues}")

We propagate the first ten datapoints using our estimated models:

In [ ]:
traj_standard = standard_model.transform(data)
traj_exact = exact_model.transform(data)

A scatter plot reveals that all data points get mapped to the same two wells and the propagator was approximated almost perfectly.

In [ ]:
plt.scatter(*data.T, alpha=.5, marker='x', label='ground truth')
plt.scatter(*np.real(traj_standard)[::5].T, marker='*',alpha=.5, label='standard DMD')
plt.scatter(*np.real(traj_exact)[::5].T, alpha=.1, label='exact DMD')
plt.legend();

## Extended DMD

For people already familiar with the method and interface, the [API docs](../api/generated/deeptime.decomposition.EDMD.rst).

Similarly to DMD, extended DMD (EDMD) <cite data-cite="nbdmd-williams2015data">(Williams, 2015)</cite> is a data-driven method to approximate the Koopman operator. The Koopman operator $\mathcal{K}$ describes how an observable of an (potentially stochastic) dynamical system evolves in time. In a discrete-time setting with $x_{t+1} = T(x_t)$ and an observable $g$, the Koopman operator is defined as

$$
(\mathcal{K} g)(x) = g(T(x)).
$$

Given observation data $X$ and $Y$, where $y_i = T(x_i)$ and a basis set of functions or observables

$$
B = \{ \psi_1,\psi_2,\ldots,\psi_k \},
$$

one can define the transformed data $\Psi_X$ and $\Psi_Y$ with 

$$(\Psi_*)_i = (\psi_1(*_i),\ldots, \psi_k(*_i))^\top, $$

yielding

$$
\Psi_Y^\top = \Psi_X^\top K.
$$

Here, $K^\top\in\mathbb{R}^{k\times k}$ acting from the left is the projection of the true Koopman operator $\mathcal{K}$ into the basis $B$.

Usually one performs rank-truncation or has over-/underdetermined systems. In such cases, $K$ is determined via the pseudo-inverse $K=\Psi_Y\Psi_X^\dagger$.

In case of a basis $B$ which just contains the identity, EDMD reduces back to DMD.

To demonstrate EDMD and deeptime's API, we consider a [two-dimensional triple-well potential](../api/generated/deeptime.data.triple_well_2d.rst#deeptime.data.triple_well_2d) <cite data-cite="nbdmd-schutte2013metastability">(Schütte, 2013)</cite>. It is defined on $[-2,2]\times [-1, 2]$ and looks like the following:

In [ ]:
import numpy as np

x = np.linspace(-2, 2, num=100)
y = np.linspace(-1, 2, num=100)
XX, YY = np.meshgrid(x, y)
V = 3*np.exp(-(XX**2) - (YY - 1/3)**2) - 3*np.exp(-XX**2 - (YY - 5/3)**2) \
    - 5*np.exp(-(XX-1)**2 - YY**2) - 5*np.exp(-(XX+1)**2 - YY**2) \
    + (2/10)*XX**4 + (2/10)*(YY-1/3)**4

plt.contourf(x, y, V, levels=np.linspace(-4.5, 4.5, 20), cmap='coolwarm');

Inside this potential landscape we integrate particles subject to the stochastic differential equation

$$ \mathrm{d}X_t = \nabla V(X_t) \mathrm{d}t + \sigma(t, X_t)\mathrm{d}W_t $$

with $\sigma=1.09$, using an Euler-Maruyama integrator.

In [ ]:
import deeptime as dt

system = dt.data.triple_well_2d()
traj = system.trajectory(x0=[[-1, 0]], n_evaluations=200, seed=66)

plt.contourf(x, y, V, levels=np.linspace(-4.5, 4.5, 20), cmap='coolwarm')
plt.plot(*traj.T, c='black', lw=.5);

To approximate the Koopman operator, we first select 25000 random points inside the domain and integrate them for 10000 steps under an integration step of $h=10^{-5}$:

In [ ]:
N = 25000
state = np.random.RandomState(seed=42)
X = np.stack([state.uniform(-2, 2, size=(N,)), state.uniform(-1, 2, size=(N,))]).T
Y = system(X, n_jobs=8)

Now we pick a basis $B$ to be all monomials up to and including degree $\mathrm{deg}=10$.

In [ ]:
basis = dt.basis.Monomials(10)

Using this basis, we can fit an EDMD estimator and obtain the corresponding model.

In [ ]:
edmd_estimator = dt.decomposition.EDMD(basis, n_eigs=8)
edmd_model = edmd_estimator.fit((X, Y)).fetch_model()

We can obtain the dominant eigenvalues associated to processes in the system.

In [ ]:
plt.plot(np.real(edmd_model.eigenvalues), 'x')
plt.title('Dominant eigenvalues');

We project our input data $X$ using the eigenfunctions:

In [ ]:
phi = np.real(edmd_model.transform(X))

# normalize
for i in range(len(edmd_model.eigenvalues)):
    phi[:, i] = phi[:, i] / np.max(np.abs(phi[:, i]))

And visualize the first four eigenfunctions. They contain information about metastable regions and slow transitions.

In [ ]:
fig = plt.figure(figsize=(8, 8))
gs = fig.add_gridspec(ncols=2, nrows=2)

ax = fig.add_subplot(gs[0, 0])
ax.scatter(*X.T, c=phi[:, 0], cmap='coolwarm')
ax.set_title('1st eigenfunction')

ax = fig.add_subplot(gs[0, 1])
ax.scatter(*X.T, c=phi[:, 1], cmap='coolwarm')
ax.set_title('2nd eigenfunction')

ax = fig.add_subplot(gs[1, 0])
ax.scatter(*X.T, c=phi[:, 2], cmap='coolwarm')
ax.set_title('3rd eigenfunction')

ax = fig.add_subplot(gs[1, 1])
ax.scatter(*X.T, c=phi[:, 3], cmap='coolwarm')
ax.set_title('4th eigenfunction');

Using a clustering can reveal temporally coherent structures.

In [ ]:
kmeans = dt.clustering.Kmeans(n_clusters=6, n_jobs=4).fit(np.real(phi)).fetch_model()
c = kmeans.transform(np.real(phi))

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1.scatter(*X.T, c=c)
ax1.set_title(r"$t = 0$")
ax1.set_aspect('equal')
ax1.set_xlim([-2.5, 2.5])
ax1.set_ylim([-1.5, 2.5])
ax1.contour(x, y, V, levels=np.linspace(-4.5, 4.5, 20), colors='black')

ax2.scatter(*Y.T, c=c)
ax2.set_title(r"$t = 0.1$")
ax2.set_aspect('equal')
ax2.set_xlim([-2.5, 2.5])
ax2.set_ylim([-1.5, 2.5])
ax2.contour(x, y, V, levels=np.linspace(-4.5, 4.5, 20), colors='black');